In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [ ]:
def cross_entropy(y,y_pred):
    return -np.mean(np.dot(y.T , np.log(y_pred)) + np.dot((1 - y.T) , np.log(1 - y_pred)))

class ANN:
    def __init__(self,hidden_num_neurons):
        # Input layer (8 features) → Hidden layer (4 neurons)
        self.W1 =  np.random.randn(8,hidden_num_neurons)
        self.b1 =  np.zeros((1,hidden_num_neurons))
        
        # Hidden layer (4 neurons) → Output layer (1 neuron)
        self.W2 = np.random.randn(hidden_num_neurons,1)
        self.b2 = np.zeros((1,1))
    

    def perceptron(self,x,w,b):
        return np.dot(x,w)+b
    
    def sigmoid(self, x): 
        return  1/(1+np.exp(-x))
    
    
    def forward_Pass(self, X):
        # Hidden layer computation (use ReLU)
        self.z1 =  	self.perceptron(X,self.W1,self.b1)	
        self.a1 =  	np.maximum(0,self.z1)  # ReLU activation, hidden layer's output
        
        # Output layer computation (use sigmoid)
        self.z2 = self.perceptron(self.a1,self.W2,self.b2) 
        self.a2 = self.sigmoid(self.z2) 		# Output (sigmoid applied to z2)
        return self.a2
    
    def backward(self, X, y,y_pred, learning_rate):
        m = X.shape[0]  # Number of samples
        
        # Output layer error
        dZ2 =  y_pred-y         # Hint: Derivative of binary cross-entropy loss (error at the output layer)
        dW2 =  np.dot(self.a1.T,dZ2)/m     # Hint: Use np.dot with proper dimensions (for output layer)
        db2 =  np.sum(dZ2,axis=0)/m  # Hint: Sum along axis 0 (for output layer)
        
        # Hidden layer error (ReLU derivative is 1 where z1 > 0)
        dZ1 = np.multiply(np.dot(dZ2,self.W2.T) , (self.z1>0))
        dW1 = np.dot(X.T,dZ1)/m
        db1 = np.sum(dZ1,axis=0)/m
        
        # Update weights and biases
        self.W1 -= learning_rate*dW1
        self.b1 -= learning_rate*db1
        self.W2 -= learning_rate*dW2
        self.b2 -= learning_rate*db2

    def predict(self, X, threshold=0.5):
        prob = self.forward_Pass(X)
        return (prob > threshold)

    
    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            y_pred = self.forward_Pass(X)
            loss = cross_entropy(y,y_pred)
            self.backward(X, y,y_pred, learning_rate)
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    


In [ ]:
data = pd.read_csv('dataset/diabetes.csv')

X = data.iloc[:, :-1].values  # Features
y = data.iloc[:, -1].values   # Labels 

# Split into train (70%) and test (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train=np.matrix(y_train).T
nn = ANN(hidden_num_neurons=80)
nn.train(X_train,y_train, epochs=1000, learning_rate=0.9)

# Predict and calculate accuracy
y_pred = nn.predict(X_test)
accuracy = np.mean(y_pred.flatten() == y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
